# INTRODUCTION AND OUTLINE #

This project creates predictions for every match for the rest of the season for any of the top 5 European Leagues, based on optimised goal, xG and betting line ratings. Defining the league variable by one of the names listed above it allows you to choose which league you would like to look at.

These predictions are then used to create a league table with predicted number of points, goals scored and goals conceded at the end of the season.
It also contains predictions for each teams likelihood to finish 1st, Top 4, Top 6 and Bottom 3, derived from running Monte Carlo simulations on the rest of the seasons fixtures.

The document is split up into the following sections

1. Understat Fixture Scraper
2. Data Clean Up
3. TotalCorner Dataset Import and Clean Up
4. Dataset Merging
5. Home Advantage Factor
6. Team Performance Dataframe
7. Finding Optimal Parameters For Assessing Teams Performance
8. Defining Optimal Parameters and Testing
9. Applying Optimal Parameters to Future Fixtures
10. Assessing Draw Factor in Historical Results
11. Calculating a Draw Adjustment Factor
10. Current Season Table
11. End of Season Table Prediction
12. Monte Carlo Simulations for Rest of Season
13. Limitations and Further Improvements

### PACKAGE IMPORTS ###

In [1]:
import pandas as pd
import understatapi
import numpy as np
import tabulate
from itertools import product
from scipy.stats import poisson,skellam
from sklearn.metrics import mean_absolute_error, mean_squared_error

### FIXTURE SCRAPER ###
This section scrapes fixture data from understat for a given league and combines into a single data frame

In [2]:
# League to import

#EPL
#La_Liga
#Serie_A
#Bundesliga
#Ligue_1

league = 'EPL'

# Seasons to import

seasons = range(2015, 2025)

# Defines league variables

league_data = {
    'EPL': {
        'league_tc': 'EnglandPremierLeague',
        'n_teams_div': 20
    },
    'La_Liga': {
        'league_tc': 'SpainLaLiga',
        'n_teams_div': 20
    },
        'Serie_A': {
        'league_tc': 'ItalySerieA',
        'n_teams_div': 20
    },
        'Bundesliga': {
        'league_tc': 'GermanyBundesligaI',
        'n_teams_div': 18
    },
        'Ligue_1': {
        'league_tc': 'FranceLigue1',
        'n_teams_div': 18
    }
}

# Defines additional variables for league

league_info = league_data[league]

league_tc = league_info['league_tc']

n_teams_div = league_info['n_teams_div']

# Sets up client

client = understatapi.UnderstatClient()

# Sets up list to store each season 

raw_data = []

# Loop to scrape data for each season, define 'season' column and append seasons to raw_data list

for season in seasons:
    data = client.league(league=league).get_match_data(season=str(season))
    df = pd.DataFrame(data)
    df['season'] = f"{season}/{season + 1}"
    raw_data.append(df)

# Combine all the data into a single DataFrame

fixtures_data = pd.concat(raw_data, ignore_index=True)

### DATA CLEANUP ###
This section reformats the data from a json format, drops unneccessary columns, converts the datetime column datatype and renames some columns

In [3]:
# Fixes formatting

dict_columns = ['h', 'a', 'goals', 'xG', 'forecast']

for col in dict_columns:
    
    formatted_df = pd.json_normalize(fixtures_data[col])
    
    formatted_df.columns = [f"{col}_{key}" for key in formatted_df.columns]
    
    fixtures_data = pd.concat([fixtures_data, formatted_df], axis=1)


# Removes redundant columns

fixtures_data.drop(['h', 'a', 'goals', 'xG', 'forecast', 'id', 'h_id', 
                    'h_short_title', 'a_id', 'a_short_title', 'forecast_w', 
                    'forecast_d', 'forecast_l'], axis = 1, inplace=True)

   
# Formats date column and renames columns

fixtures_data['datetime'] = pd.to_datetime(fixtures_data['datetime']).dt.strftime('%Y-%m-%d')

fixtures_data = fixtures_data.rename(columns  = {'h_title':'Home', 'a_title':'Away', 'datetime': 'Date'})


In [4]:
print(fixtures_data.head(10).to_markdown())

|    | isResult   | Date       | season    | Home                 | Away            |   goals_h |   goals_a |     xG_h |     xG_a |
|---:|:-----------|:-----------|:----------|:---------------------|:----------------|----------:|----------:|---------:|---------:|
|  0 | True       | 2015-08-08 | 2015/2016 | Manchester United    | Tottenham       |         1 |         0 | 0.627539 | 0.6746   |
|  1 | True       | 2015-08-08 | 2015/2016 | Bournemouth          | Aston Villa     |         0 |         1 | 0.876106 | 0.782253 |
|  2 | True       | 2015-08-08 | 2015/2016 | Everton              | Watford         |         2 |         2 | 0.604226 | 0.557892 |
|  3 | True       | 2015-08-08 | 2015/2016 | Leicester            | Sunderland      |         4 |         2 | 2.56803  | 1.45946  |
|  4 | True       | 2015-08-08 | 2015/2016 | Norwich              | Crystal Palace  |         1 |         3 | 1.13076  | 2.10975  |
|  5 | True       | 2015-08-08 | 2015/2016 | Chelsea              | Swansea 

We now have a clean dataframe with all past and future fixtures, with goals and xG performances for each team

### TOTALCORNER DATASET IMPORT AND CLEAN UP ###
Here we are importing another dataset that contains match odds for all fixtures, we will merge this with our understat data to get odds for each fixture

In [5]:
# Imports totalcorner fixtures
fixtures_tc = pd.read_csv(f"{league_tc}.csv")

# Formats date column

fixtures_tc['Date'] = pd.to_datetime(fixtures_tc['Date'], format='%d.%m.%Y').dt.date

# Creates unique match identifier and reduces to one row per match

fixtures_tc['matchid'] = fixtures_tc['Date'].astype(str) +"-"+ fixtures_tc["Home"] +"-"+ fixtures_tc["Away"]

fixtures_tc = fixtures_tc.drop_duplicates(subset='matchid', keep='first')

# Converts AH Line to a supremacy level

def hcap_conv(string):
    if isinstance(string, str):
        string = string[:-1]
        string = string[6:]
    
    string = str(string)
    
    if ',' in string:
        parts = string.split(', ')
        value1 = float(parts[0])
        value2 = float(parts[1])
        result = (value1 + value2) / 2
    else:
        result = float(string)
    
    return result

fixtures_tc['hcaplevel'] = fixtures_tc['AH.Line'].apply(hcap_conv)

# Removes margin from AH prices

fixtures_tc['price_home'] = (fixtures_tc['AH.Home.Odds'] + fixtures_tc['AH.Away.Odds']) / fixtures_tc['AH.Away.Odds']

fixtures_tc['price_away'] = (fixtures_tc['AH.Home.Odds'] + fixtures_tc['AH.Away.Odds']) / fixtures_tc['AH.Home.Odds']

# Rounds digits for merging

fixtures_tc[['price_home', 'price_away', 'Goal.O.Odds', 'Goal.U.Odds']] = fixtures_tc[['price_home', 'price_away', 'Goal.O.Odds', 'Goal.U.Odds']].round(2)

# Imports look up tables to convert odds to goals and supremacy values

sup_conv_table = pd.read_csv("C:/Users/Luke/Desktop/Researchdocs/sup_conversion.csv")

goals_conv_table = pd.read_csv("C:/Users/Luke/Desktop/Researchdocs/goallines.csv")

# Merges to get goal and sup values

fixtures_tc = pd.merge(fixtures_tc, sup_conv_table, on=["hcaplevel", "price_home"], how="left")

fixtures_tc = pd.merge(fixtures_tc, goals_conv_table, on=["Goal.Line", "Goal.O.Odds"], how="left")


C:\Users\Luke\AppData\Local\Temp\ipykernel_1100\2356984188.py:2: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  fixtures_tc = pd.read_csv(f"{league_tc}.csv")


### DATASET MERGING ###

Now we have our two datasets in a clean and manageble format we can combine them. Here we identify the team names and fix any name discrepancies between the datasets. We then merge the datasets to add goals and supremacy totals derived from betting lines to the fixtures, goals and xG from understat. 

In [6]:
# Identifies teams to harmonise names

sorted_teams_us = sorted(fixtures_data['Home'].unique())

sorted_teams_tc = sorted(fixtures_tc['Home'].unique())

# Converts names

name_conv = {'Parma Calcio 1913':'Parma', 'Inter':'Inter Milan', 'SPAL 2013':'Spal',
             'Wolverhampton Wanderers':'Wolverhampton', 'West Bromwich Albion':'West Brom', 'Sheffield United':'Sheff Utd',
             'Manchester City':'Man City', 'Manchester United':'Man Utd', 'Newcastle United':'Newcastle', 'Nottingham Forest':'Nottm Forest',
             'Alaves':'CD Alaves', 'Athletic Club':'Athletic Bilbao', 'Real Valladolid': 'Valladolid', 'SD Huesca':'Huesca', 
             'Borussia M.Gladbach':'Borussia M\'gladbach', 'FC Cologne':'Cologne', 'FC Heidenheim':'Heidenheim','Freiburg': 'SC Freiburg',
             'Hamburger SV':'Hamburg', 'Hoffenheim':'TSG Hoffenheim', 'Ingolstadt':'FC Ingolstadt', 'Mainz 05':'Mainz',
             'Nuernberg':'Nurnberg', 'RasenBallsport Leipzig':'RB Leipzig', 'Schalke 04':'Schalke', 'St. Pauli':'St Pauli',
             'Fortuna Duesseldorf':'Fortuna Dusseldorf', 'Ajaccio':'AC Ajaccio', 'GFC Ajaccio':'Ajaccio GFCA',
             'Paris Saint Germain':'PSG', 'Saint-Etienne':'St Etienne'
             }

fixtures_data['Home'] = fixtures_data['Home'].replace(name_conv)

fixtures_data['Away'] = fixtures_data['Away'].replace(name_conv)

# Creates unique match identifier

fixtures_data['matchid'] = fixtures_data['Date'].astype(str) +"-"+ fixtures_data["Home"] +"-"+ fixtures_data["Away"]

# Merges to attach odds data

fixtures_data = fixtures_data.merge(
    fixtures_tc[['matchid', 'adjsup', 'Gls']],
    on=['matchid'],
    how='left'
)

# Renames columns

fixtures_data = fixtures_data.rename(columns={'Gls': 'asian_total_goals'})

fixtures_data = fixtures_data.rename(columns={'adjsup': 'asian_sup'})

# Adds missing data for rogue Bundesliga match

fixtures_data.loc[fixtures_data['matchid'] == '2025-02-09-Holstein Kiel-Bochum', ['isResult', 'goals_h', 'goals_a', 'xG_h', 'xG_a']] = [True, 2, 2, 1.37, 2.05]

In [7]:
print(fixtures_data.head(10).to_markdown())

|    | isResult   | Date       | season    | Home        | Away           |   goals_h |   goals_a |     xG_h |     xG_a | matchid                            |   asian_sup |   asian_total_goals |
|---:|:-----------|:-----------|:----------|:------------|:---------------|----------:|----------:|---------:|---------:|:-----------------------------------|------------:|--------------------:|
|  0 | True       | 2015-08-08 | 2015/2016 | Man Utd     | Tottenham      |         1 |         0 | 0.627539 | 0.6746   | 2015-08-08-Man Utd-Tottenham       |        0.86 |                2.75 |
|  1 | True       | 2015-08-08 | 2015/2016 | Bournemouth | Aston Villa    |         0 |         1 | 0.876106 | 0.782253 | 2015-08-08-Bournemouth-Aston Villa |        0.69 |                2.65 |
|  2 | True       | 2015-08-08 | 2015/2016 | Everton     | Watford        |         2 |         2 | 0.604226 | 0.557892 | 2015-08-08-Everton-Watford         |        0.71 |                2.65 |
|  3 | True       | 2015-

The dataset now contains a unique match identifier and goals and supremacy values based on asian handicap betting lines

### HOME ADVANTAGE FACTOR ###
In order to make predictions for future matches we need to identify the strength of each team, as well as introduce a metric that accounts for home advantage. As part of this process we also remove data that was played behind closed doors during the pandemic. Playing in stadiums without fans had an effect on home advantage, and altered the relationship between home team strength, away team strength and home advantage on the outcome of matches. Most leagues did not return to full stadiums until at least some way through the 2021/2022, so I have excluded all data from 1st March 2020 until the end of that season.

In [8]:
# Filters for past matches

fixtures_data_past = fixtures_data[fixtures_data['isResult'] == True].copy()

# Converts datatypes

to_float = ['xG_h', 'xG_a']

to_int = ['goals_h', 'goals_a']

fixtures_data_past[to_float] = fixtures_data_past[to_float].astype(float)

fixtures_data_past[to_int] = fixtures_data_past[to_int].astype(int)

# Removes matches played behind closed doors

covid_period_start = '2020-03-01'

covid_period_end = '2022-07-01'
                                                                                                         
fixtures_data_past = fixtures_data_past[(fixtures_data_past['Date'] < covid_period_start) | (fixtures_data_past['Date'] > covid_period_end)]                                                                                                                

# Finds average home and away goals over this period and defines home advantage factor 

fixtures_data_past_homeadv = pd.DataFrame({
    'homegoals': [fixtures_data_past['goals_h'].mean()],
    'awaygoals': [fixtures_data_past['goals_a'].mean()]
})

fixtures_data_past_homeadv['total_goals'] = fixtures_data_past_homeadv['homegoals'] + fixtures_data_past_homeadv['awaygoals']

fixtures_data_past_homeadv['homeadv'] = fixtures_data_past_homeadv['homegoals'] - fixtures_data_past_homeadv['awaygoals']

fixtures_data_past_homeadv['homeadv_pc'] = fixtures_data_past_homeadv['homeadv'] / fixtures_data_past_homeadv['total_goals']

home_adv = float(fixtures_data_past_homeadv['homeadv_pc'].iloc[0])

fixtures_data_past['HA'] = home_adv

In [9]:
season_counts = fixtures_data_past['season'].value_counts().sort_index()
print(season_counts)
print(home_adv)

season
2015/2016    380
2016/2017    380
2017/2018    380
2018/2019    380
2019/2020    276
2022/2023    380
2023/2024    380
2024/2025    271
Name: count, dtype: int64
0.1096822060096451


We now have a dataset that excludes all matches played behind closed doors and a home advantage factor we can use to make predictions for future matches

### TEAM PERFORMANCE DATAFRAME ###
Here we create a team focused dataframe that identifes team performace in each match based on goals, xG and team goal expectancies derived from betting lines

In [10]:
# Adds extra columns for analysis

fixtures_data_past['AsianHomeGoals'] = (fixtures_data_past['asian_sup'] + fixtures_data_past['asian_total_goals']) / 2

fixtures_data_past['AsianAwayGoals'] = fixtures_data_past['asian_total_goals'] - fixtures_data_past['AsianHomeGoals']

fixtures_data_past['total_goals'] = fixtures_data_past['goals_h'] + fixtures_data_past['goals_a']

fixtures_data_past['total_xG'] = fixtures_data_past['xG_h'] + fixtures_data_past['xG_a']

# Drops rows with missing odds data

fixtures_data_past = fixtures_data_past.dropna(subset = 'AsianHomeGoals')

fixtures_data_past = fixtures_data_past.dropna(subset = 'AsianAwayGoals')

# Creates team focused dataframe for team strength

fixtures_data_past_home = fixtures_data_past[['matchid','Date', 'Home', 'goals_h', 'goals_a', 'xG_h', 'xG_a', 'AsianHomeGoals', 'AsianAwayGoals', 'total_goals', 'total_xG', 'asian_total_goals']].copy()

fixtures_data_past_home['home_or_away'] = 'Home'

fixtures_data_past_away = fixtures_data_past[['matchid','Date', 'Away', 'goals_a', 'goals_h', 'xG_a', 'xG_h', 'AsianAwayGoals', 'AsianHomeGoals', 'total_goals', 'total_xG', 'asian_total_goals']].copy()

fixtures_data_past_away['home_or_away'] = 'Away'

fixtures_data_past_home = fixtures_data_past_home.rename(columns={'Home': 'Team', 
                                                                          'goals_h': 'GoalsScored', 'goals_a': 'GoalsConceded',
                                                                          'xG_h': 'xG_for', 'xG_a': 'xG_conc',
                                                                          'AsianHomeGoals': 'Asian_for', 'AsianAwayGoals': 'Asian_conc',
                                                                          })

fixtures_data_past_away = fixtures_data_past_away.rename(columns={'Away': 'Team', 
                                                                          'goals_a': 'GoalsScored', 'goals_h': 'GoalsConceded', 
                                                                          'xG_a': 'xG_for', 'xG_h': 'xG_conc',
                                                                          'AsianAwayGoals': 'Asian_for', 'AsianHomeGoals': 'Asian_conc',})


fixtures_data_team_strength = pd.concat([fixtures_data_past_home, fixtures_data_past_away], ignore_index=True)

fixtures_data_team_strength = fixtures_data_team_strength.sort_values(['Date', 'matchid'], ascending=False).reset_index()

In [11]:
print(fixtures_data_team_strength.head(10).to_markdown())

|    |   index | matchid                         | Date       | Team         |   GoalsScored |   GoalsConceded |   xG_for |   xG_conc |   Asian_for |   Asian_conc |   total_goals |   total_xG |   asian_total_goals | home_or_away   |
|---:|--------:|:--------------------------------|:-----------|:-------------|--------------:|----------------:|---------:|----------:|------------:|-------------:|--------------:|-----------:|--------------------:|:---------------|
|  0 |    2785 | 2025-02-27-West Ham-Leicester   | 2025-02-27 | West Ham     |             2 |               0 | 1.70437  |  0.289578 |       1.725 |        1.015 |             2 |    1.99395 |                2.74 | Home           |
|  1 |    5571 | 2025-02-27-West Ham-Leicester   | 2025-02-27 | Leicester    |             0 |               2 | 0.289578 |  1.70437  |       1.015 |        1.725 |             2 |    1.99395 |                2.74 | Away           |
|  2 |    2782 | 2025-02-26-Tottenham-Man City   | 2025-02-26 | Tott

We now have a team focused dataframe with one row for each match per team, this can be used to create rolling averages of teams performance that we can use to create ratings to predict future performances.

### FINDING OPTIMAL PARAMETERS FOR ASSESSING TEAMS PERFORMANCE ###
To create predictions for future matches I want to derive expected goals and supremacy values for each match for the rest of the season. To derive these I will use team attack strength and defence strength ratings to predict the number of goals scored by each team in each match. To get these predictions I will use the formulae:

#### Home_Pred_Goals = ((Home_AS * Away_DS) / DS_avg) + (HA_factor / 2) ####
#### Away_Pred_Goals = ((Away_AS* Home_DS ) / DS_avg) - (HA_factor / 2) ####

Home_AS, Away_AS, Home_DS and Away_DS are measures of the past attack and defense performance for each team, based on historic goals, xG performaces and betting lines.

DS_avg is the average amount of goals conceded per match by a single team over the same time period, HA_factor is the home advantage factor we identified earlier.

In order to optimise the attack and defence strength values for each team I want to find the best combination of n_matches, goals, xG and betting line ratings. This function tests a range of n_matches and weighted combinations of each strength measure to find the optimal combination of these parameters.

I am assessing each combination by measuring the mean absolute error of the predicted goals and supremacy for each match compared to asian_total_goals and asian_sup. These values are derived from historic betting lines and are assumed to be efficient at KO. 

In [12]:
# Splits data into train and test sets

fixtures_data_past_train = fixtures_data_past[fixtures_data_past['Date'] < '2023-08-01'].copy()

fixtures_data_past_test = fixtures_data_past[fixtures_data_past['Date'] > '2023-08-01'].copy()

# Defines ranges for parameters

n_matches_range = range(5, 30)
goals_wgt_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
xG_wgt_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
asian_wgt_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

cols_to_avg_team = ['GoalsScored', 'GoalsConceded', 'xG_for', 'xG_conc', 'Asian_for', 'Asian_conc']

cols_to_avg_league = ['total_goals', 'total_xG', 'asian_total_goals']

# Function to find maes of different parameters

def calculate_predictions(n_matches, goals_wgt, xG_wgt, asian_wgt, fixtures, team_strength):

    if goals_wgt + xG_wgt + asian_wgt != 1:
        raise ValueError("Weights must sum to 1.")
    
    # Team averages for each n
    
    for col in cols_to_avg_team:
        team_strength[f'{col}_roll'] = (
            team_strength
            .groupby('Team', group_keys=False)[col]
            .transform(lambda x: x.shift(-1)[::-1].rolling(window=n_matches, min_periods=n_matches).mean()[::-1])
        )
    # League averages for each n * n_teams_div
    
    for col in cols_to_avg_league:
        team_strength[f'{col}_roll'] = (
            team_strength[col]
            .transform(lambda x: x.shift(-1)[::-1].rolling(window=(n_matches * n_teams_div), min_periods=(n_matches * n_teams_div)).mean()[::-1])
        )
    
    # Merges for team strength with past fixtures for home team
    
    fixtures = fixtures.merge(
        team_strength[team_strength['home_or_away'] == 'Home'][['matchid', 'Team', 'GoalsScored_roll', 'GoalsConceded_roll', 'xG_for_roll', 'xG_conc_roll', 'Asian_for_roll', 'Asian_conc_roll', 'total_goals_roll', 'total_xG_roll', 'asian_total_goals_roll']],
        left_on=['Home', 'matchid'],
        right_on=['Team', 'matchid'],
        how='left'
    ).rename(
        columns={'GoalsScored_roll': 'Home_GoalsScored_roll', 'GoalsConceded_roll': 'Home_GoalsConceded_roll', 
                 'xG_for_roll': 'Home_xG_for_roll', 'xG_conc_roll': 'Home_xG_conc_roll', 
                 'Asian_for_roll': 'Home_Asian_for_roll', 'Asian_conc_roll': 'Home_Asian_conc_roll'}
    ).drop(columns=['Team'])
    
    # Merges for team strength with past fixtures for away team
    
    fixtures = fixtures.merge(
        team_strength[team_strength['home_or_away'] == 'Away'][['matchid', 'Team', 'GoalsScored_roll', 'GoalsConceded_roll', 'xG_for_roll', 'xG_conc_roll', 'Asian_for_roll', 'Asian_conc_roll']],
        left_on=['Away', 'matchid'],
        right_on=['Team', 'matchid'],
        how='left'
    ).rename(
        columns={'GoalsScored_roll': 'Away_GoalsScored_roll', 'GoalsConceded_roll': 'Away_GoalsConceded_roll', 
                 'xG_for_roll': 'Away_xG_for_roll', 'xG_conc_roll': 'Away_xG_conc_roll', 
                 'Asian_for_roll': 'Away_Asian_for_roll', 'Asian_conc_roll': 'Away_Asian_conc_roll'}
    ).drop(columns=['Team'])
    
    # Calculates av goals per team

    fixtures['avg_ds_goals'] = fixtures['total_goals_roll'] / 2

    # Drops any rows with missing rolling avs due to small sample size
    
    fixtures = fixtures.dropna()
    
    # Calculates weighted home, away team AS and DS based on weights
    
    fixtures['weight_AS_home'] = (
        fixtures['Home_GoalsScored_roll'] * goals_wgt +
        fixtures['Home_xG_for_roll'] * xG_wgt +
        fixtures['Home_Asian_for_roll'] * asian_wgt
    )
    
    fixtures['weight_DS_home'] = (
        fixtures['Home_GoalsConceded_roll'] * goals_wgt +
        fixtures['Home_xG_conc_roll'] * xG_wgt +
        fixtures['Home_Asian_conc_roll'] * asian_wgt
    )
    
    fixtures['weight_AS_away'] = (
        fixtures['Away_GoalsScored_roll'] * goals_wgt +
        fixtures['Away_xG_for_roll'] * xG_wgt +
        fixtures['Away_Asian_for_roll'] * asian_wgt
    )
    
    fixtures['weight_DS_away'] = (
        fixtures['Away_GoalsConceded_roll'] * goals_wgt +
        fixtures['Away_xG_conc_roll'] * xG_wgt +
        fixtures['Away_Asian_conc_roll'] * asian_wgt
    )
    
    # Calculates predictions based on this combination of weights
    
    fixtures['home_pred'] = (fixtures['weight_AS_home'] * fixtures['weight_DS_away']) / fixtures['avg_ds_goals']
    fixtures['away_pred'] = (fixtures['weight_AS_away'] * fixtures['weight_DS_home']) / fixtures['avg_ds_goals']
    fixtures['total_pred'] = fixtures['home_pred'] + fixtures['away_pred']
    fixtures['sup_pred'] = fixtures['home_pred'] - fixtures['away_pred']
    fixtures['Match_HA'] = fixtures['total_pred'] * fixtures['HA']
    fixtures['home_pred'] = fixtures['home_pred'] + (fixtures['Match_HA'] / 2)
    fixtures['away_pred'] = fixtures['away_pred'] - (fixtures['Match_HA'] / 2)
    fixtures['total_pred'] = fixtures['home_pred'] + fixtures['away_pred']
    fixtures['sup_pred'] = fixtures['home_pred'] - fixtures['away_pred']
    
    # Calculates MAE against asian lines for each match
    
    mae_total = mean_absolute_error(fixtures['total_pred'], fixtures['asian_total_goals'])
    mae_sup = mean_absolute_error(fixtures['sup_pred'], fixtures['asian_sup'])
    
    return mae_total, mae_sup



### DEFINING BEST PARAMETERS AND TESTING ###
In this section we run the function on a range of combination of weights and n matches and record the results of the mae for goals and supremacy in a dataframe. The parameters with the lowest weighted mae are identified and tested against the test dataset.

In [13]:
# Creates initial list

results = []

# Iterates through all combos and creates dataframe to store results

for n_matches, goals_wgt, xG_wgt, asian_wgt in product(n_matches_range, goals_wgt_range, xG_wgt_range, asian_wgt_range):
    try:
        mae_total, mae_sup = calculate_predictions(n_matches, goals_wgt, xG_wgt, asian_wgt, fixtures_data_past_train.copy(), fixtures_data_team_strength.copy())
        
        results.append({
            'n_matches': n_matches,
            'goals_wgt': goals_wgt,
            'xG_wgt': xG_wgt,
            'asian_wgt': asian_wgt,
            'mae_total': mae_total,
            'mae_sup': mae_sup
        })
    except ValueError:
        continue  
    

results_df = pd.DataFrame(results)

# Preference for finding sup diffs

results_df['comb_mae'] = (results_df['mae_total']*0.25) + (results_df['mae_sup']*0.75)

results_df = results_df.sort_values('comb_mae').reset_index()

best_params = [results_df['n_matches'].iloc[0], results_df['goals_wgt'].iloc[0], results_df['xG_wgt'].iloc[0], results_df['asian_wgt'].iloc[0]]

# Test params against test data

mae_total_test, mae_sup_test = calculate_predictions(
    n_matches=best_params[0],
    goals_wgt=best_params[1],
    xG_wgt=best_params[2],
    asian_wgt=best_params[3],
    fixtures=fixtures_data_past_test.copy(),
    team_strength=fixtures_data_team_strength.copy()
)


In [14]:
print(results_df.head(10).to_markdown())

print(best_params)

print(mae_total_test, mae_sup_test)

|    |   index |   n_matches |   goals_wgt |   xG_wgt |   asian_wgt |   mae_total |   mae_sup |   comb_mae |
|---:|--------:|------------:|------------:|---------:|------------:|------------:|----------:|-----------:|
|  0 |     449 |          19 |         0.1 |      0.2 |         0.7 |    0.214683 |  0.2305   |   0.226546 |
|  1 |     416 |          18 |         0.1 |      0.1 |         0.8 |    0.213092 |  0.231548 |   0.226934 |
|  2 |     448 |          19 |         0.1 |      0.1 |         0.8 |    0.211995 |  0.231996 |   0.226996 |
|  3 |     352 |          16 |         0.1 |      0.1 |         0.8 |    0.212613 |  0.232007 |   0.227158 |
|  4 |     417 |          18 |         0.1 |      0.2 |         0.7 |    0.215676 |  0.231174 |   0.2273   |
|  5 |     384 |          17 |         0.1 |      0.1 |         0.8 |    0.213022 |  0.232178 |   0.227389 |
|  6 |     481 |          20 |         0.1 |      0.2 |         0.7 |    0.215356 |  0.231657 |   0.227582 |
|  7 |     456 |   

We have now identified the combination of n_matches goals_wgt, xG_wgt and asian_wgt that have historically minimised the error between our predicted goals and supremacy and the goals and supremacy implied by the betting lines for each match. We will use the parameters to make future predictions.

### APPLYING OPTIMAL PARAMETERS TO FUTURE FIXTURES ###
In this section we use the optimal params identified above to get the ratings to predict future matches. We then use these ratings combined with our home advantage factor to make predictions for every match for the rest of the season.

In [15]:
### Current team averages

current_team_strength = fixtures_data_team_strength.copy()

best_n_matches = best_params[0]

# Team averages for best_n_matches

for col in cols_to_avg_team:
    current_team_strength[f'{col}_roll'] = (
    current_team_strength
   .groupby('Team', group_keys=False)[col]
   .transform(lambda x: x[::-1].rolling(window=best_n_matches, min_periods=best_n_matches).mean()[::-1])
)

# League averages for best_n_matches * n_teams_div
    
for col in cols_to_avg_league:
    current_team_strength[f'{col}_roll'] = (
    current_team_strength[col]
    .transform(lambda x: x[::-1].rolling(window=(best_n_matches * n_teams_div), min_periods=(best_n_matches * n_teams_div)).mean()[::-1])
)

# Filters for most recent ratings

current_team_strength = current_team_strength.drop_duplicates(subset='Team', keep='first')

# Filters for this seasons teams

current_season_teams = current_team_strength[(current_team_strength['Date'] > '2024-07-01')]

current_season_teams = current_season_teams['Team'].unique()

current_team_strength = current_team_strength[current_team_strength['Team'].isin(current_season_teams)]

# Filters relevant columns

current_team_AS_DS = current_team_strength[['Team', 'GoalsScored_roll', 'xG_for_roll', 'Asian_for_roll', 'GoalsConceded_roll', 'xG_conc_roll', 'Asian_conc_roll', 'total_goals_roll']].copy()

# Assigns weights based on optimal params

goals_wgt = best_params[1]

xG_wgt = best_params[2]

asian_wgt = best_params[3]

# Applies weights to get optimised team AS and DS ratings

current_team_AS_DS['weight_AS'] = (current_team_AS_DS['GoalsScored_roll'] * goals_wgt) + (current_team_AS_DS['xG_for_roll'] * xG_wgt) + (current_team_AS_DS['Asian_for_roll'] * asian_wgt)

current_team_AS_DS['weight_DS'] = (current_team_AS_DS['GoalsConceded_roll'] * goals_wgt) + (current_team_AS_DS['xG_conc_roll'] * xG_wgt) + (current_team_AS_DS['Asian_conc_roll'] * asian_wgt)

current_team_AS_DS['DS_avg'] = current_team_AS_DS['weight_DS'].mean()

# Upcoming fixtures

fixtures_data_future = fixtures_data[fixtures_data['isResult'] == False].copy()

fixtures_data_future = fixtures_data_future[fixtures_data_future['Date'] > '2024-08-01']

fixtures_data_future = fixtures_data_future[['Date', 'Home', 'Away', ]]

# Adds AS and DS to upcoming fixtures

fixtures_data_future = fixtures_data_future.merge(
    current_team_AS_DS[['Team', 'weight_AS', 'weight_DS', 'DS_avg']],
    left_on='Home',
    right_on='Team',
    how='left'
).rename(columns={'weight_AS': 'Home_AS', 'weight_DS': 'Home_DS'}).drop(columns=['Team'])

fixtures_data_future = fixtures_data_future.merge(
    current_team_AS_DS[['Team', 'weight_AS', 'weight_DS']],
    left_on='Away',
    right_on='Team',
    how='left'
).rename(columns={'weight_AS': 'Away_AS', 'weight_DS': 'Away_DS'}).drop(columns=['Team'])

# Predicted match goals and supremacy for rest of the season

fixtures_data_future['HA'] = home_adv

fixtures_data_future['Home_Pred_Goals'] = (fixtures_data_future['Home_AS'] * fixtures_data_future['Away_DS']) / fixtures_data_future['DS_avg']

fixtures_data_future['Away_Pred_Goals'] = (fixtures_data_future['Away_AS'] * fixtures_data_future['Home_DS']) / fixtures_data_future['DS_avg']

fixtures_data_future['Total_Pred_Goals'] = fixtures_data_future['Home_Pred_Goals'] + fixtures_data_future['Away_Pred_Goals']

fixtures_data_future['Pred_Sup'] = fixtures_data_future['Home_Pred_Goals'] - fixtures_data_future['Away_Pred_Goals']

# Adds home advantage factor

fixtures_data_future['Total_HA'] = fixtures_data_future['Total_Pred_Goals'] * fixtures_data_future['HA']

fixtures_data_future['Home_Pred_Goals'] = fixtures_data_future['Home_Pred_Goals'] + (fixtures_data_future['Total_HA'] / 2)

fixtures_data_future['Away_Pred_Goals'] = fixtures_data_future['Away_Pred_Goals'] - (fixtures_data_future['Total_HA'] / 2)

fixtures_data_future['Total_Pred_Goals'] = fixtures_data_future['Home_Pred_Goals'] + fixtures_data_future['Away_Pred_Goals']

fixtures_data_future['Pred_Sup'] = fixtures_data_future['Home_Pred_Goals'] - fixtures_data_future['Away_Pred_Goals']

fixtures_data_future.drop(['Home_AS', 'Home_DS', 'Home_DS', 'DS_avg', 'Away_AS', 'Away_DS', 'HA', 
                    'Total_HA'], axis = 1, inplace=True)

current_team_AS_DS.drop(['GoalsScored_roll', 'xG_for_roll', 'Asian_for_roll', 'GoalsConceded_roll', 'xG_conc_roll', 'Asian_conc_roll', 'total_goals_roll', 
                    'DS_avg'], axis = 1, inplace=True)

In [16]:
print(current_team_AS_DS.head(20).to_markdown())

|    | Team           |   weight_AS |   weight_DS |
|---:|:---------------|------------:|------------:|
|  0 | West Ham       |     1.2557  |    1.81154  |
|  1 | Leicester      |     1.09491 |    2.03166  |
|  2 | Tottenham      |     1.77826 |    1.81516  |
|  3 | Man City       |     2.08791 |    1.37751  |
|  4 | Nottm Forest   |     1.43658 |    1.49388  |
|  5 | Arsenal        |     1.94054 |    0.884448 |
|  6 | Man Utd        |     1.57259 |    1.44823  |
|  7 | Ipswich        |     1.10878 |    2.06217  |
|  8 | Liverpool      |     2.31823 |    1.12605  |
|  9 | Newcastle      |     1.82478 |    1.445    |
| 10 | Brentford      |     1.58471 |    1.68262  |
| 11 | Everton        |     1.15435 |    1.45762  |
| 12 | Wolverhampton  |     1.23564 |    1.72123  |
| 13 | Fulham         |     1.46622 |    1.37357  |
| 14 | Crystal Palace |     1.47851 |    1.42111  |
| 15 | Aston Villa    |     1.59038 |    1.53166  |
| 16 | Chelsea        |     2.09854 |    1.35472  |
| 17 | South

In [17]:
print(fixtures_data_future.head(10).to_markdown())

|    | Date       | Home           | Away        |   Home_Pred_Goals |   Away_Pred_Goals |   Total_Pred_Goals |   Pred_Sup |
|---:|:-----------|:---------------|:------------|------------------:|------------------:|-------------------:|-----------:|
|  0 | 2025-03-08 | Nottm Forest   | Man City    |           1.43807 |          1.80803  |            3.2461  |  -0.369967 |
|  1 | 2025-03-08 | Brighton       | Fulham      |           1.59924 |          1.21877  |            2.81801 |   0.380477 |
|  2 | 2025-03-08 | Crystal Palace | Ipswich     |           2.10288 |          0.841825 |            2.9447  |   1.26105  |
|  3 | 2025-03-08 | Liverpool      | Southampton |           3.75578 |          0.491887 |            4.24767 |   3.26389  |
|  4 | 2025-03-08 | Brentford      | Aston Villa |           1.72372 |          1.52572  |            3.24944 |   0.198007 |
|  5 | 2025-03-08 | Wolverhampton  | Everton     |           1.27911 |          1.13287  |            2.41198 |   0.146236 |


The first table show the curent weighted attack and defence ratings for each team in the league. These are used to make the predictions for upcoming games in the second table.

### ASSESSING DRAW FACTOR IN HISTORICAL DATA ###

To make predictions for future matches we will use a poisson distribution to predict the outcome of each match based on their predicted goals identified above. One weakness of using a poisson distribution is that it assumes goals are statistically independent of each other. In reality goals scored by teams in any match will be influenced by the current score and overall gamestate. Historically an unrefined poisson model has underestimated draws and low scoring scorelines in general.

This section compares the proportion of draws observed in  historical data with how many draws would be predicted by a standard poisson model. It also shows the difference between poisson predicted scorelines and observed scorelines in low scoring matches.

In [18]:
# Observed scorelines for all past matches

scorelines = [(home_goals, away_goals) for home_goals in range(11) for away_goals in range(11)]

total_matches = len(fixtures_data_past_train)

observed_probs = {
    scoreline: len(fixtures_data_past_train[(fixtures_data_past_train['goals_h'] == scoreline[0]) & (fixtures_data_past_train['goals_a'] == scoreline[1])]) / total_matches
    for scoreline in scorelines
}

# Observed scoreline probs matrix

max_goals = 11

observed_probs_matrix = np.zeros((max_goals, max_goals))

for (home_goals, away_goals), prob in observed_probs.items():
    observed_probs_matrix[home_goals, away_goals] = prob

observed_probs_matrix = pd.DataFrame(observed_probs_matrix)

# Calculate predicted scorelines for all past matches

predicted_probs = {scoreline: 0 for scoreline in scorelines}

for _, row in fixtures_data_past_train.iterrows():
    home_exp = row['AsianHomeGoals']
    away_exp = row['AsianAwayGoals']

    for scoreline in scorelines:
        home_goals, away_goals = scoreline
        prob = poisson.pmf(home_goals, home_exp) * poisson.pmf(away_goals, away_exp)
        predicted_probs[scoreline] += prob

# Average scoreline probabilities

predicted_probs = {scoreline: prob / len(fixtures_data_past_train) for scoreline, prob in predicted_probs.items()}

# Predicted scoreline probs matrix

max_goals = 11

predicted_probs_matrix = np.zeros((max_goals, max_goals))

for (home_goals, away_goals), prob in predicted_probs.items():
    predicted_probs_matrix[home_goals, away_goals] = prob

predicted_probs_matrix = pd.DataFrame(predicted_probs_matrix)

scorelinediffs = predicted_probs_matrix - observed_probs_matrix

# Predicted outcome probabilities

predicted_outcomes = {"home_win": 0, "draw": 0, "away_win": 0}

for scoreline, prob in predicted_probs.items():
    home_goals, away_goals = scoreline
    
    if home_goals > away_goals:  
        predicted_outcomes["home_win"] += prob
    elif home_goals == away_goals:  
        predicted_outcomes["draw"] += prob
    else:
        predicted_outcomes["away_win"] += prob

# Observed outcomes probabilities

observed_outcomes = {"home_win": 0, "draw": 0, "away_win": 0}

for _, row in fixtures_data_past_train.iterrows():
    if row['goals_h'] > row['goals_a']: 
        observed_outcomes["home_win"] += 1
    elif row['goals_h'] == row['goals_a']:
        observed_outcomes["draw"] += 1
    else:
        observed_outcomes["away_win"] += 1

observed_outcomes = {key: value / total_matches for key, value in observed_outcomes.items()}


In [19]:
print(predicted_outcomes)

print(observed_outcomes)

print(scorelinediffs.iloc[:3, :3])


{'home_win': 0.4555414895447065, 'draw': 0.22855282999256232, 'away_win': 0.3158757671033817}
{'home_win': 0.46053246146660437, 'draw': 0.237739374124241, 'away_win': 0.3017281644091546}
          0         1         2
0 -0.004696  0.013383  0.003468
1 -0.004128  0.002096 -0.003149
2 -0.006164 -0.001719 -0.007922


Here we can see how 1X2 outcomes predicted by a poisson model compare to our observed outcomes. It also shows differences in observed scorelines and poisson predicted scorelines in low scoring matches.

### CALCULATING A DRAW ADJUSTMENT FACTOR ###
This section creates adjustment factors for low scoring matches, we can use this to refine our predictions for future matches

In [20]:
# Calculates adjustment factors for drawing scorelines

low_scorelines = [(0, 0), (1, 1), (2, 2), (1, 0), (0, 1), (2, 1), (1, 2)]
low_scorelines_observed = {scoreline: observed_probs[scoreline] for scoreline in low_scorelines}
low_scorelines_predicted = {scoreline: predicted_probs[scoreline] for scoreline in low_scorelines}

adjustment_factors = {
    scoreline: low_scorelines_observed[scoreline] / low_scorelines_predicted[scoreline]
    for scoreline in low_scorelines
}

# Function to calculate probabilities with adjusted scorelines for matches in training set

def calculate_adjusted_probabilities(row):
    home_goal_expectation = row['AsianHomeGoals']
    away_goal_expectation = row['AsianAwayGoals']
    
    home_probs = poisson.pmf(range(11), home_goal_expectation)
    away_probs = poisson.pmf(range(11), away_goal_expectation)
    
    probability_matrix = np.outer(home_probs, away_probs)
    
    adjusted_probability_matrix = probability_matrix.copy()
    for (i, j), factor in adjustment_factors.items():
        adjusted_probability_matrix[i, j] *= factor
    
    adjusted_probability_matrix /= adjusted_probability_matrix.sum()
    
    home = np.sum(np.tril(adjusted_probability_matrix, -1))  
    draw = np.sum(np.diag(adjusted_probability_matrix))     
    away = np.sum(np.triu(adjusted_probability_matrix, 1))
    
    return pd.Series([home, draw, away], index=['home_pc', 'draw_pc', 'away_pc'])

fixtures_data_past_train[['home_pc', 'draw_pc', 'away_pc']] = fixtures_data_past_train.apply(calculate_adjusted_probabilities, axis=1)

fixtures_data_past_train[['home_win_price', 'draw_price', 'away_win_price']] = 1 / fixtures_data_past_train[['home_pc', 'draw_pc', 'away_pc']]

fixtures_data_past_train = fixtures_data_past_train.round(2)

# Sense checking adjusted predicted outcomes

predicted_outcomes_adjusted = {"home_win": 0, "draw": 0, "away_win": 0}

predicted_outcomes_adjusted['home_win'] = fixtures_data_past_train['home_pc'].sum() / len(fixtures_data_past_train)

predicted_outcomes_adjusted['draw'] = fixtures_data_past_train['draw_pc'].sum() / len(fixtures_data_past_train)

predicted_outcomes_adjusted['away_win'] = fixtures_data_past_train['away_pc'].sum() / len(fixtures_data_past_train)


In [21]:
print(predicted_outcomes)

print(predicted_outcomes_adjusted)

print(observed_outcomes)

{'home_win': 0.4555414895447065, 'draw': 0.22855282999256232, 'away_win': 0.3158757671033817}
{'home_win': 0.4580102755721625, 'draw': 0.23771134983652498, 'away_win': 0.30428304530593175}
{'home_win': 0.46053246146660437, 'draw': 0.237739374124241, 'away_win': 0.3017281644091546}


This shows the effect that the scoreline adjustments have made in getting our 1X2 predictions closer to observed historical results.

### CURRENT SEASON TABLE ###

This section creates a league table based on results so far this season

In [23]:
# Sets up current table 

current_season_table = pd.DataFrame(columns=['Team','MP', 'W','D','L','GF','GA','GD','Pts'])

current_season_table['Team'] = current_season_teams

current_season_table[['MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts']] = 0

currentseason_past = fixtures_data[fixtures_data['Date'] > '2024-07-01'].copy()

currentseason_past = currentseason_past[currentseason_past['isResult'] == True]

currentseason_past[to_float] = currentseason_past[to_float].astype(float)

currentseason_past[to_int] = currentseason_past[to_int].astype(int)

# Adds values to table based on results

for index, row in currentseason_past.iterrows():
    home, away = row['Home'], row['Away']
    goals_h, goals_a = row['goals_h'], row['goals_a']

    home_index = current_season_table[current_season_table['Team'] == home].index[0]
    away_index = current_season_table[current_season_table['Team'] == away].index[0]
    
    current_season_table.at[home_index, 'MP'] += 1
    current_season_table.at[away_index, 'MP'] += 1

    if goals_h > goals_a:
        current_season_table.at[home_index, 'Pts'] += 3
        current_season_table.at[home_index, 'W'] += 1
        current_season_table.at[away_index, 'L'] += 1
    elif goals_h < goals_a: 
        current_season_table.at[away_index, 'Pts'] += 3
        current_season_table.at[away_index, 'W'] += 1
        current_season_table.at[home_index, 'L'] += 1
    else: 
        current_season_table.at[home_index, 'Pts'] += 1
        current_season_table.at[away_index, 'Pts'] += 1
        current_season_table.at[home_index, 'D'] += 1
        current_season_table.at[away_index, 'D'] += 1

    current_season_table.at[home_index, 'GF'] += goals_h
    current_season_table.at[home_index, 'GA'] += goals_a
    current_season_table.at[home_index, 'GD'] += (goals_h - goals_a)
    
    current_season_table.at[away_index, 'GF'] += goals_a
    current_season_table.at[away_index, 'GA'] += goals_h
    current_season_table.at[away_index, 'GD'] += (goals_a - goals_h)


current_season_table = current_season_table.sort_values(['Pts', 'GD', 'GF'], ascending=False)

In [24]:
print(current_season_table.to_markdown())

|    | Team           |   MP |   W |   D |   L |   GF |   GA |   GD |   Pts |
|---:|:---------------|-----:|----:|----:|----:|-----:|-----:|-----:|------:|
|  8 | Liverpool      |   28 |  20 |   7 |   1 |   66 |   26 |   40 |    67 |
|  5 | Arsenal        |   27 |  15 |   9 |   3 |   51 |   23 |   28 |    54 |
|  4 | Nottm Forest   |   27 |  14 |   6 |   7 |   44 |   33 |   11 |    48 |
|  3 | Man City       |   27 |  14 |   5 |   8 |   53 |   37 |   16 |    47 |
| 16 | Chelsea        |   27 |  13 |   7 |   7 |   52 |   36 |   16 |    46 |
|  9 | Newcastle      |   27 |  13 |   5 |   9 |   46 |   38 |    8 |    44 |
| 19 | Bournemouth    |   27 |  12 |   7 |   8 |   45 |   32 |   13 |    43 |
| 18 | Brighton       |   27 |  11 |  10 |   6 |   44 |   39 |    5 |    43 |
| 13 | Fulham         |   27 |  11 |   9 |   7 |   40 |   36 |    4 |    42 |
| 15 | Aston Villa    |   28 |  11 |   9 |   8 |   40 |   45 |   -5 |    42 |
| 10 | Brentford      |   27 |  11 |   5 |  11 |   48 |   43 |  

### END OF SEASON TABLE PREDICTION ###

This section makes predictions for all future matches based on the predicted goals for each match. It adjusts the probability of each outcome to account for the increased prevalence of draws and low scoring matches. It then sums the predicted number of wins, draw, losses and goals for each team, adds these values to the current table and sorts to give a prediction of what the league table will look like at the end of the season.

In [25]:
# Function to get 1X2 predictions for future matches, with adjustments for low scoring draws

def calculate_adjusted_probabilities_future(row):
    home_goal_expectation = row['Home_Pred_Goals']
    away_goal_expectation = row['Away_Pred_Goals']
    
    home_probs = poisson.pmf(range(11), home_goal_expectation)
    away_probs = poisson.pmf(range(11), away_goal_expectation)
    
    probability_matrix = np.outer(home_probs, away_probs)
    
    adjusted_probability_matrix = probability_matrix.copy()
    for (i, j), factor in adjustment_factors.items():
        adjusted_probability_matrix[i, j] *= factor
    
    adjusted_probability_matrix /= adjusted_probability_matrix.sum()
    
    home = np.sum(np.tril(adjusted_probability_matrix, -1))  
    draw = np.sum(np.diag(adjusted_probability_matrix))     
    away = np.sum(np.triu(adjusted_probability_matrix, 1))
    
    return pd.Series([home, draw, away], index=['home_pc', 'draw_pc', 'away_pc'])

fixtures_data_future[['home_pc', 'draw_pc', 'away_pc']] = fixtures_data_future.apply(calculate_adjusted_probabilities_future, axis=1)

fixtures_data_future[['home_win_price', 'draw_price', 'away_win_price']] = 1 / fixtures_data_future[['home_pc', 'draw_pc', 'away_pc']]

# Sets up future table prediction

current_season_table_predictions = current_season_table.copy()

columns_to_float = ['Pts', 'W', 'D', 'L', 'GF', 'GA', 'GD']

current_season_table_predictions[columns_to_float] = current_season_table_predictions[columns_to_float].astype(float)

# Adds points, goals and matches played to current table

for index, row in fixtures_data_future.iterrows():
    home, away = row['Home'], row['Away']
    home_pc, draw_pc, away_pc, Exp_HG, Exp_AG = row['home_pc'], row['draw_pc'], row['away_pc'], row['Home_Pred_Goals'], row['Away_Pred_Goals']

    home_index = current_season_table_predictions[current_season_table_predictions['Team'] == home].index[0]
    away_index = current_season_table_predictions[current_season_table_predictions['Team'] == away].index[0]
    
    current_season_table_predictions.at[home_index, 'MP'] += 1
    
    current_season_table_predictions.at[home_index, 'Pts'] += 3 * home_pc
    current_season_table_predictions.at[home_index, 'Pts'] += 1 * draw_pc
    
    current_season_table_predictions.at[home_index, 'W'] += 1 * home_pc
    current_season_table_predictions.at[home_index, 'D'] += 1 * draw_pc
    current_season_table_predictions.at[home_index, 'L'] += 1 * away_pc
    
    current_season_table_predictions.at[away_index, 'MP'] += 1
    
    current_season_table_predictions.at[away_index, 'Pts'] += 3 * away_pc
    current_season_table_predictions.at[away_index, 'Pts'] += 1 * draw_pc
    
    current_season_table_predictions.at[away_index, 'W'] += 1 * away_pc
    current_season_table_predictions.at[away_index, 'D'] += 1 * draw_pc
    current_season_table_predictions.at[away_index, 'L'] += 1 * home_pc
    
    current_season_table_predictions.at[home_index, 'GF'] += Exp_HG
    current_season_table_predictions.at[home_index, 'GA'] += Exp_AG
    current_season_table_predictions.at[home_index, 'GD'] += (Exp_HG - Exp_AG)
    
    current_season_table_predictions.at[away_index, 'GF'] += Exp_AG
    current_season_table_predictions.at[away_index, 'GA'] += Exp_HG
    current_season_table_predictions.at[away_index, 'GD'] += (Exp_AG - Exp_HG)

current_season_table_predictions = current_season_table_predictions.sort_values(['Pts', 'GD', 'GF'], ascending=False)

current_season_table_predictions = current_season_table_predictions.round(2)

In [26]:
print(current_season_table_predictions.to_markdown())

|    | Team           |   MP |     W |     D |     L |    GF |    GA |     GD |   Pts |
|---:|:---------------|-----:|------:|------:|------:|------:|------:|-------:|------:|
|  8 | Liverpool      |   38 | 26.49 |  8.81 |  2.7  | 90.01 | 36.33 |  53.68 | 88.29 |
|  5 | Arsenal        |   38 | 21.75 | 11.39 |  4.86 | 72.45 | 32.63 |  39.83 | 76.65 |
|  3 | Man City       |   38 | 20.39 |  7.26 | 10.35 | 76.94 | 50.27 |  26.68 | 68.43 |
| 16 | Chelsea        |   38 | 18.83 |  9.25 |  9.91 | 74.74 | 50.64 |  24.09 | 65.76 |
|  4 | Nottm Forest   |   38 | 18.33 |  8.72 | 10.95 | 60.62 | 48.83 |  11.79 | 63.7  |
|  9 | Newcastle      |   38 | 18.28 |  7.46 | 12.26 | 66.12 | 53.03 |  13.09 | 62.31 |
| 19 | Bournemouth    |   38 | 16.78 |  9.47 | 11.75 | 63.76 | 48.19 |  15.57 | 59.8  |
| 18 | Brighton       |   38 | 15.63 | 12.56 |  9.81 | 62.38 | 55.45 |   6.93 | 59.45 |
| 13 | Fulham         |   38 | 14.86 | 11.7  | 11.44 | 55.38 | 52.64 |   2.74 | 56.27 |
| 15 | Aston Villa    |   38 | 1

This is a prediction of how the league table will look at the end of the season.

### MONTE CARLO SIMULATION OF REST OF SEASON ###
This section runs a Monte Carlo simulation to calculate the likelihood of each team finishing 1st, top 4, top 6 and bottom 3 in the league at the end of the season.

In [27]:
# Defines simulation variables

N = 1000
teams = current_season_teams
num_teams = len(teams)

results = pd.DataFrame(0, index=teams, columns=['1st', 'Top 4', 'Top 6', 'Bottom 3'])
results.index.name = "Team"

# Function to simulate a single match

def simulate_match(home_pc, draw_pc, away_pc):
    outcome = np.random.choice(['home', 'draw', 'away'], p=[home_pc, draw_pc, away_pc])
    if outcome == 'home':
        return 3, 0  
    elif outcome == 'draw':
        return 1, 1  
    else:
        return 0, 3  

pts_table = current_season_table[['Team', 'Pts']].copy()

pts_table['Pts']  = pts_table['Pts'].astype(float)

# Runs the simulations

for sim in range(N):

    sim_table = pts_table.copy()
    
    # Simulates remaining matches and add predicted points to league table
    for _, row in fixtures_data_future.iterrows():
        home, away = row['Home'], row['Away']
        home_pc, draw_pc, away_pc = row['home_pc'], row['draw_pc'], row['away_pc']
        
        home_pts, away_pts = simulate_match(home_pc, draw_pc, away_pc)
        
        home_index = sim_table[sim_table['Team'] == home].index[0]
        sim_table.at[home_index, 'Pts'] += home_pts

        away_index = sim_table[sim_table['Team'] == away].index[0]
        sim_table.at[away_index, 'Pts'] += away_pts

    # Sorts table
    sim_table = sim_table.sort_values('Pts', ascending=False).reset_index(drop=True)
    
    # Adds 1 to results for each sim in each bucket for each team
    results.loc[sim_table.iloc[0]['Team'], '1st'] += 1
    for i, team in enumerate(sim_table['Team']):
        if i < 4:
            results.loc[team, 'Top 4'] += 1
        if i < 6:
            results.loc[team, 'Top 6'] += 1
        if i >= num_teams - 3:
            results.loc[team, 'Bottom 3'] += 1

league_predictions = results / N

table_predictions_probs = pd.merge(current_season_table_predictions, league_predictions, on=["Team"], how="left")

In [28]:
print(table_predictions_probs.to_markdown())

|    | Team           |   MP |     W |     D |     L |    GF |    GA |     GD |   Pts |   1st |   Top 4 |   Top 6 |   Bottom 3 |
|---:|:---------------|-----:|------:|------:|------:|------:|------:|-------:|------:|------:|--------:|--------:|-----------:|
|  0 | Liverpool      |   38 | 26.49 |  8.81 |  2.7  | 90.01 | 36.33 |  53.68 | 88.29 | 0.989 |   1     |   1     |      0     |
|  1 | Arsenal        |   38 | 21.75 | 11.39 |  4.86 | 72.45 | 32.63 |  39.83 | 76.65 | 0.011 |   0.995 |   0.999 |      0     |
|  2 | Man City       |   38 | 20.39 |  7.26 | 10.35 | 76.94 | 50.27 |  26.68 | 68.43 | 0     |   0.803 |   0.955 |      0     |
|  3 | Chelsea        |   38 | 18.83 |  9.25 |  9.91 | 74.74 | 50.64 |  24.09 | 65.76 | 0     |   0.525 |   0.887 |      0     |
|  4 | Nottm Forest   |   38 | 18.33 |  8.72 | 10.95 | 60.62 | 48.83 |  11.79 | 63.7  | 0     |   0.346 |   0.754 |      0     |
|  5 | Newcastle      |   38 | 18.28 |  7.46 | 12.26 | 66.12 | 53.03 |  13.09 | 62.31 | 0     |  

The final table shows the predicted league table with the probabilities of each team finishing 1st, Top 4, Top 6 and Bottom 3

### LIMITATIONS AND FURTHER IMPROVEMENTS ###

- The ratings do not take into account short term fluctuations in form due to player transfers, injuries, suspensions and manager changes.
- Historic betting lines are usually the best predictor of future results, however they are a function of previous xG and goals performance. Combining these factors to create ratings runs the risk of them being cross correlated.
- The model does not currently have a way of assessing the performance of newly promoted teams. A more advanced model could take their performance in the lower division and apply a promotion factor that re-evaluates their performance in context of the higher division. After a few matches cross referencing betting lines against other teams can give a picture of how the market rates the newly promoted teams compared to their opponents.
- The rolling averages used to assess teams historic performance could include a weighting factor that weights more recent preformances more than older performances. This approach runs the risk of skewing team performance ratings if they have a particularly tough run of fixtures, however that could be mitigated by assessing teams performance in the context of their opponents strength.
- Assessing the model based on its mae against historic betting lines is just one way of measuring performance. Other techniques could include looking at the root mean squared error (rmse) if we wanted to prioritise minimising big differences, or mean absolute percentage error (MAPE) if we were worried about errors with high goal and supremacy lines skewing the results.
- Another way to make the model more robust would be to split the dataset into smaller subsets and train the model multiple times, each time using a different subset as the validation set. Checking the model has similar results across these subsets would protect against overfitting the model.
- The predictions will be used to find value in betting markets, another way to assess its performance is to find historic examples of when the model would have found value in the market and assess whether betting on these would have been profitable.